# Introdução

Na primeira etapa dos experimentos de finetuning, nós fizemos um total de 20 experimentos, variando:
 - tamanho do modelo (`t5-base` e `t5-small`)
 - como tratar caracteres `\n`: substituindo por espaço (` `) ou pipes (`|`)
 - modelo treinado sobre dataset com apenas uma tarefa ou todas (concatenação de datasets de todas tarefas)

Aqui vou salvar todas predições numa pasta, pra evitar de ter que ficar rodando toda hora.

# Initial setup

In [1]:
%load_ext autoreload
%autoreload 2

# Loading expriment data from Neptune

In [2]:
# Loading project data
import neptune
project = neptune.init('marcospiau/final-project-ia376j-1')
experiments = project.get_experiments()
display(experiments)

[Experiment(FIN-27),
 Experiment(FIN-28),
 Experiment(FIN-29),
 Experiment(FIN-30),
 Experiment(FIN-33),
 Experiment(FIN-34),
 Experiment(FIN-35),
 Experiment(FIN-36),
 Experiment(FIN-37),
 Experiment(FIN-38),
 Experiment(FIN-39),
 Experiment(FIN-40),
 Experiment(FIN-41),
 Experiment(FIN-42),
 Experiment(FIN-43),
 Experiment(FIN-44),
 Experiment(FIN-45),
 Experiment(FIN-46),
 Experiment(FIN-47),
 Experiment(FIN-48)]

In [45]:
leaderboard = project.get_leaderboard()
leaderboard.head()

,id,name,created,finished,owner,notes,running_time,size,tags,channel_best_model_path,...,property_max_epochs,property_min_epochs,property_param__generate_max_length,property_param__learning_rate,property_param__optimizer,property_param__t5_model_prefix,property_param__t5_tokenizer_prefix,property_patience,property_str_replace_newlines,property_task_train
0,FIN-27,t5-base_all_tasks_concat_newlines_as_pipes,2021-01-02 06:49:33.609000+00:00,2021-01-02 15:58:31.521000+00:00,marcospiau,,32937,13603174,"[t5-base, all_tasks_concat, t5_ocr_baseline]",/home/marcospiau/final_project_ia376j/checkpoi...,...,200,20,512,0.001,adafactor,t5-base,t5-base,50,|,all_tasks_concat
1,FIN-28,t5-base_all_tasks_concat_newlines_as_spaces,2021-01-02 15:58:50.605000+00:00,2021-01-02 22:02:48.959000+00:00,marcospiau,,21838,9068745,"[t5-base, all_tasks_concat, t5_ocr_baseline]",/home/marcospiau/final_project_ia376j/checkpoi...,...,200,20,512,0.001,adafactor,t5-base,t5-base,50,,all_tasks_concat
2,FIN-29,t5-small_all_tasks_concat_newlines_as_pipes,2021-01-02 22:03:04.051000+00:00,2021-01-03 01:38:50.889000+00:00,marcospiau,,12946,5292147,"[t5-small, all_tasks_concat, t5_ocr_baseline]",/home/marcospiau/final_project_ia376j/checkpoi...,...,200,20,512,0.001,adafactor,t5-small,t5-small,50,|,all_tasks_concat
3,FIN-30,t5-small_all_tasks_concat_newlines_as_spaces,2021-01-03 01:39:06.081000+00:00,2021-01-03 03:38:56.588000+00:00,marcospiau,,7190,2940305,"[t5-small, all_tasks_concat, t5_ocr_baseline]",/home/marcospiau/final_project_ia376j/checkpoi...,...,200,20,512,0.001,adafactor,t5-small,t5-small,50,,all_tasks_concat
4,FIN-33,t5-base_extract_company_newlines_as_pipes,2021-01-03 04:02:04.774000+00:00,2021-01-03 05:35:03.390000+00:00,marcospiau,,5578,2446937,"[t5-base, extract_company, t5_ocr_baseline]",/home/marcospiau/final_project_ia376j/checkpoi...,...,200,20,512,0.001,adafactor,t5-base,t5-base,50,|,extract_company


In [47]:
[x.name for x in experiments]

['t5-base_all_tasks_concat_newlines_as_pipes',
 't5-base_all_tasks_concat_newlines_as_spaces',
 't5-small_all_tasks_concat_newlines_as_pipes',
 't5-small_all_tasks_concat_newlines_as_spaces',
 't5-base_extract_company_newlines_as_pipes',
 't5-base_extract_company_newlines_as_spaces',
 't5-small_extract_company_newlines_as_pipes',
 't5-small_extract_company_newlines_as_spaces',
 't5-base_extract_total_newlines_as_pipes',
 't5-base_extract_total_newlines_as_spaces',
 't5-small_extract_total_newlines_as_pipes',
 't5-small_extract_total_newlines_as_spaces',
 't5-base_extract_address_newlines_as_pipes',
 't5-base_extract_address_newlines_as_spaces',
 't5-small_extract_address_newlines_as_pipes',
 't5-small_extract_address_newlines_as_spaces',
 't5-base_extract_date_newlines_as_pipes',
 't5-base_extract_date_newlines_as_spaces',
 't5-small_extract_date_newlines_as_pipes',
 't5-small_extract_date_newlines_as_spaces']

In [48]:
from src.data.sroie import get_all_keynames_from_dir
from src.data.sroie import get_dataloaders_dict_from_datasets_dict
from src.data.sroie import get_tasks_functions_maps
from src.data.sroie.t5_ocr_baseline import get_datasets_dict_from_task_functions_map
from src.data.sroie.t5_ocr_baseline import get_default_preprocessing_functions
from src.models.t5_ocr_baseline import T5OCRBaseline
from functools import partial
from tqdm import tqdm
import gc
import pytorch_lightning as pl
import gin
import tempfile
import multiprocessing as mp
import os
import torch
import pandas as pd

gin.enter_interactive_mode()
gin.clear_config()

In [ ]:
gin.parse_config_file(
    '/home/marcospiau/final_project_ia376j/src/models/gin/defaults.gin',
    skip_unknown=True
)
with gin.config_scope('train_sroie'):
    train_keynames = get_all_keynames_from_dir()

with gin.config_scope('validation_sroie'):
    val_keynames = get_all_keynames_from_dir()

gin.clear_config()

In [176]:
123

123

In [149]:
# experiments_to_eval = project.get_experiments(tag='all_tasks_concat')
experiments = dict((x.id, x) for x in project.get_experiments())

In [140]:
train_preds = dict() 
val_preds = dict()

In [177]:
123

123

In [ ]:
for experiment in experiments.values():
    print(120 * '*')
    print(f'Evaluating experiment {experiment.id}')
    
    gin.clear_config()
#     Loading experiment data
    experiment_channels = experiment.get_channels()
    experiment_properties = experiment.get_properties()
    
#     Loading best checkpoint
    model = T5OCRBaseline.load_from_checkpoint(
        experiment_channels['best_model_path'].y
    )
    model.eval()
    model.freeze()
    model.to('cuda');
    
#     Datasets and dataloaders
    with tempfile.TemporaryDirectory() as tmp_folder:
        experiment.download_artifact('gin_operative_config.gin', tmp_folder)
        gin.parse_config_file(
            os.path.join(tmp_folder, 'gin_operative_config.gin'),
            skip_unknown=True
        )

    with gin.config_scope('sroie_t5_baseline'):
        task_functions_maps = get_tasks_functions_maps()

    train_datasets = get_datasets_dict_from_task_functions_map(
        keynames=train_keynames, tasks_functions_maps=task_functions_maps)
    val_datasets = get_datasets_dict_from_task_functions_map(
        keynames=val_keynames, tasks_functions_maps=task_functions_maps)

    loader_kwargs = {
        'num_workers': mp.cpu_count(),
        'shuffle': False,
        'pin_memory': True
    }
    train_dataloaders = get_dataloaders_dict_from_datasets_dict(
        datasets_dict=train_datasets, dataloader_kwargs=loader_kwargs)
    val_dataloaders = get_dataloaders_dict_from_datasets_dict(
        datasets_dict=val_datasets, dataloader_kwargs=loader_kwargs)
    
#     Predictions for training and validation data
    assert train_dataloaders.keys() == val_dataloaders.keys()
    
    train_preds[experiment.id] = extract_all_fields_from_model(
        model, train_dataloaders, train_keynames)
    val_preds[experiment.id] = extract_all_fields_from_model(
        model, val_dataloaders, val_keynames)

    del model
    torch.cuda.empty_cache()
    gc.collect()

************************************************************************************************************************
Evaluating experiment FIN-27


Some weights of the model checkpoint at t5-base were not used when initializing T5ForConditionalGeneration: ['decoder.block.0.layer.1.EncDecAttention.relative_attention_bias.weight']
- This IS expected if you are initializing T5ForConditionalGeneration from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing T5ForConditionalGeneration from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Extracting field company: 100%|██████████| 25/25 [00:16<00:00,  1.49it/s]


************************************************************************************************************************
Evaluating experiment FIN-28


Some weights of the model checkpoint at t5-base were not used when initializing T5ForConditionalGeneration: ['decoder.block.0.layer.1.EncDecAttention.relative_attention_bias.weight']
- This IS expected if you are initializing T5ForConditionalGeneration from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing T5ForConditionalGeneration from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Extracting field company: 100%|██████████| 25/25 [00:18<00:00,  1.33it/s]


************************************************************************************************************************
Evaluating experiment FIN-29


...

Some weights of the model checkpoint at t5-base were not used when initializing T5ForConditionalGeneration: ['decoder.block.0.layer.1.EncDecAttention.relative_attention_bias.weight']
- This IS expected if you are initializing T5ForConditionalGeneration from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing T5ForConditionalGeneration from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Extracting field company: 100%|██████████| 25/25 [00:16<00:00,  1.49it/s]


************************************************************************************************************************
Evaluating experiment FIN-34


Some weights of the model checkpoint at t5-base were not used when initializing T5ForConditionalGeneration: ['decoder.block.0.layer.1.EncDecAttention.relative_attention_bias.weight']
- This IS expected if you are initializing T5ForConditionalGeneration from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing T5ForConditionalGeneration from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Extracting field company: 100%|██████████| 25/25 [00:16<00:00,  1.49it/s]


************************************************************************************************************************
Evaluating experiment FIN-35


Some weights of the model checkpoint at t5-small were not used when initializing T5ForConditionalGeneration: ['decoder.block.0.layer.1.EncDecAttention.relative_attention_bias.weight']
- This IS expected if you are initializing T5ForConditionalGeneration from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing T5ForConditionalGeneration from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Extracting field address:   0%|          | 0/14 [00:00<?, ?it/s]

In [152]:
id(train_preds['FIN-27'])

140350976007136

In [ ]:
extract_all_fields_from_model()

In [179]:
val_preds.keys()

dict_keys(['FIN-27', 'FIN-28', 'FIN-29', 'FIN-30', 'FIN-33', 'FIN-34', 'FIN-35', 'FIN-36', 'FIN-37', 'FIN-38', 'FIN-39', 'FIN-40', 'FIN-41', 'FIN-42', 'FIN-43', 'FIN-44', 'FIN-45', 'FIN-46', 'FIN-47', 'FIN-48'])

In [130]:
def extract_base_keyname(keyname):
    return os.path.basename(keyname)

def get_single_empty_prediction(keyname):
    return dict.fromkeys(['company', 'date', 'address', 'total'], '')

def get_multiple_empty_predictions(keynames):
    base_keynames = list(map(extract_base_keyname, keynames))
    return {
        keyname: get_single_empty_prediction(keyname)
        for keyname in base_keynames
    }

def extract_all_fields_from_model(model, dataloader_dict, keynames, device='cuda'):
    preds = get_multiple_empty_predictions(keynames)
    
    for task, dataloader in dataloader_dict.items():
        if task != 'all_tasks_concat':
            field = task.replace('extract_', '')
            for batch in tqdm(
                dataloader_dict[task] ,desc=f'Extracting field {field}'):
                batch['input_ids'] = batch['input_ids'].to(device)
                preds_values = model.predict(batch)
                preds_keynames = batch['keyname']
                for key_prediction, value_prediction in zip(
                    map(extract_base_keyname, preds_keynames),
                    preds_values
                ):
                    preds[key_prediction][field] = value_prediction
    return preds

In [ ]:
field_to_extract_from_question('')

In [92]:
preds = []
for batch in tqdm(val_dataloaders['extract_date']):
    batch['input_ids'] = batch['input_ids'].to('cuda')
    
    preds.extend(model.predict(batch))

100%|██████████| 25/25 [00:17<00:00,  1.45it/s]


In [131]:
preds_val = extract_all_fields_from_model(model, val_dataloaders, val_keynames)

Extracting field company: 100%|██████████| 25/25 [00:16<00:00,  1.49it/s]


In [132]:
preds_val

{'X51006913007': {'company': 'AEON CO. (M) BHD',
  'date': '27/05/2018',
  'address': '3RD FLR, AEON TAMAN MALURI SC JLN JEJAKA, TAMAN MALURI CHERAS, 55100 KUALA LUMPUR',
  'total': '99.90'},
 'X51006913019': {'company': 'AEON CO. (M) BHD',
  'date': '11/05/2018',
  'address': '3RD FLR, AEON TAMAN MALURI SC JLN JEJAKA, TAMAN MALURI CHERAS, 55100 KUALA LUMPUR',
  'total': '476.80'},
 'X51006913023': {'company': 'UNIHAKKA INTERNATIONAL SDN BHD',
  'date': '16 MAY 2018',
  'address': '12, JALAN TAMPOI 7/4,KAWASAN PERINDUSTRIAN TAMPOI,81200 JOHOR BAHRU,JOHOR',
  'total': '$8.20'},
 'X51006913024': {'company': 'UNIHAKKA INTERNATIONAL SDN BHD',
  'date': '14 MAY 2018',
  'address': '12, JALAN TAMPOI 7/4,KAWASAN PERINDUSTRIAN TAMPOI,81200 JOHOR BAHRU,JOHOR',
  'total': '$8.90'},
 'X51006913028': {'company': 'UNIHAKKA INTERNATIONAL SDN BHD',
  'date': '07 MAY 2018',
  'address': '12, JALAN TAMPOI 7/4,KAWASAN PERINDUSTRIAN TAMPOI,81200 JOHOR BAHRU,JOHOR',
  'total': '$8.70'},
 'X51006913030': {

In [117]:
preds = get_multiple_empty_predictions(val_keynames)

for task, dataloader in val_dataloaders.items():
    if task != 'all_tasks_concat':
        field = task.replace('extract_', '')
        for batch in tqdm(
            val_dataloaders[task],desc=f'Extracting field {field}'):
            batch['input_ids'] = batch['input_ids'].to('cuda')
            preds_values = model.predict(batch)
            preds_keynames = batch['keyname']
            for key_prediction, value_prediction in zip(
                map(extract_base_keyname, preds_keynames),
                preds_values
            )
                preds[key_prediction][field] = value_prediction

Extracting field company: 100%|██████████| 25/25 [00:16<00:00,  1.50it/s]


In [129]:
preds = get_multiple_empty_predictions(val_keynames)

for task, dataloader in val_dataloaders.items():
    if task != 'all_tasks_concat':
        field = task.replace('extract_', '')
        for batch in tqdm(
            val_dataloaders[task],desc=f'Extracting field {field}'):
            batch['input_ids'] = batch['input_ids'].to('cuda')
            preds_values = model.predict(batch)
            preds_keynames = batch['keyname']
            for key_prediction, value_prediction in zip(
                map(extract_base_keyname, preds_keynames),
                preds_values
            ):
                preds[key_prediction][field] = value_prediction

SyntaxError: invalid syntax (<ipython-input-129-3d6f81a915f1>, line 14)

In [118]:
preds

{'X51006913007': {'company': 'AEON CO. (M) BHD',
  'date': '27/05/2018',
  'address': '3RD FLR, AEON TAMAN MALURI SC JLN JEJAKA, TAMAN MALURI CHERAS, 55100 KUALA LUMPUR',
  'total': '99.90'},
 'X51006913019': {'company': 'AEON CO. (M) BHD',
  'date': '11/05/2018',
  'address': '3RD FLR, AEON TAMAN MALURI SC JLN JEJAKA, TAMAN MALURI CHERAS, 55100 KUALA LUMPUR',
  'total': '476.80'},
 'X51006913023': {'company': 'UNIHAKKA INTERNATIONAL SDN BHD',
  'date': '16 MAY 2018',
  'address': '12, JALAN TAMPOI 7/4,KAWASAN PERINDUSTRIAN TAMPOI,81200 JOHOR BAHRU,JOHOR',
  'total': '$8.20'},
 'X51006913024': {'company': 'UNIHAKKA INTERNATIONAL SDN BHD',
  'date': '14 MAY 2018',
  'address': '12, JALAN TAMPOI 7/4,KAWASAN PERINDUSTRIAN TAMPOI,81200 JOHOR BAHRU,JOHOR',
  'total': '$8.90'},
 'X51006913028': {'company': 'UNIHAKKA INTERNATIONAL SDN BHD',
  'date': '07 MAY 2018',
  'address': '12, JALAN TAMPOI 7/4,KAWASAN PERINDUSTRIAN TAMPOI,81200 JOHOR BAHRU,JOHOR',
  'total': '$8.70'},
 'X51006913030': {